In [1]:
import csv
import tweepy
import ssl
import pandas as pd
import numpy as np
import openai
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 100000)

In [59]:
ssl._create_default_https_context = ssl._create_unverified_context
# Oauth keys
consumer_key = "gGRiYPUuuhoITbhGklzIO6ZvF"
consumer_secret = "jgEejdxKGqEaa8MnP02rzgCap6nbb34PknIcVIyLXfiTuDxYG6"
access_token = "410230715-kA0BPkYbnNPNHpiHpFRf5IB8ujjVGsatjvOkZJAN"
access_token_secret = "AzBSeCFs8zUFgR2eFyoNHvaJPHwrSiXjRRTsd45ieLBY0"
# Authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth ,wait_on_rate_limit=True)

In [2]:
data = pd.read_csv('/Users/ikuti/Desktop/ikuti/Other/hate-speech-and-offensive-language/data/labeled_data.csv')
data.shape

(24783, 7)

In [111]:
name = 'MrDtAFC'

In [123]:
test_tweets = []
tw = api.user_timeline(screen_name=name, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
for info in tw:
     #print("ID: {}".format(info.id))
     #print(info.created_at)
     test_tweets.append([info.created_at,info.full_text])

In [3]:
#imports for model train
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

In [4]:
## look at more data
more_data = pd.read_csv('/Users/ikuti/Desktop/ikuti/Other/hate-speech-and-offensive-language/data/more_data.csv')

In [5]:
more_data.sample()

,is_offensive,text
69244,0,adn Canwest shareholders


In [6]:
more_data.dropna( inplace = True)

In [7]:
##clean text
def clean_tweet_v2(df):
    df['text'] = df.text.str.replace('!|RT','')
    result = []
    for i in df[['text']].itertuples():
        a_ = i[1].split()
        as_ = [x.strip() for x in a_ if '@' not in x]
        asp_ = [x for x in as_ if 'http' not in x]
        result.append(' '.join([x for x in asp_ if '&#' not in x]))
    df['clean_tweets'] = result
    return df

In [8]:
more_data = clean_tweet_v2(more_data)

In [9]:
more_data.sample()

,is_offensive,text,clean_tweets
112382,1,I think a lot of bitches don't smile in their ...,I think a lot of bitches don't smile in their ...


In [10]:
#split data for training
x_train,x_test,y_train,y_test = train_test_split(more_data.clean_tweets,more_data.is_offensive,test_size = 0.2 , random_state = 23)

In [11]:
## transform text using count vectorizer
cv = CountVectorizer()
cv.fit(more_data.clean_tweets)
X_train = cv.transform(x_train)
X_test = cv.transform(x_test)

In [13]:
### transform data for xgb
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [14]:
param = {
    'eta': 0.1, 
    'max_depth': 5,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 15  # The number of training iterations

In [15]:
model = xgb.train(param, D_train, steps)

In [16]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.9440362696639357
Recall = 0.7725263520850669
Accuracy = 0.9086520206129645


In [17]:
##Naive bayes 
nb = MultinomialNB(alpha = 0.25)
nb.fit(X_train , y_train)
pred = nb.predict(X_test)
score = accuracy_score(pred,y_test) * 100

In [18]:
score

91.64090046107947

In [33]:
## decision tree with
dt_more = DecisionTreeClassifier(random_state = 0)
dt_more.fit(X_train , y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [34]:
pred_dt_m = dt_more.predict(X_test)
score_dt_m = accuracy_score(pred_dt_m , y_test) * 100

In [35]:
print("Precision = {}".format(precision_score(y_test, pred_dt_m, average='macro')))
print("Recall = {}".format(recall_score(y_test, pred_dt_m, average='macro')))

Precision = 0.9174041216428841
Recall = 0.9045670483104844


In [36]:
score_dt_m

94.4019528071603

In [37]:
confusion_matrix(pred_dt_m , y_test)

array([[28654,  1180],
       [  884,  6152]])

In [39]:
print(classification_report(y_test , pred_dt_m))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97     29538
           1       0.87      0.84      0.86      7332

    accuracy                           0.94     36870
   macro avg       0.92      0.90      0.91     36870
weighted avg       0.94      0.94      0.94     36870



In [12]:
##save model
'''
import pickle

filename = 'decision_tree.pkl'
with open(filename , 'wb') as file:
    pickle.dump(dt_more, file)
'''

In [15]:
## function to predict
def get_prediction(text , model , cv):
    ##text in list
    text = cv.transform([text])
    pred = model.predict(text)
    if pred[0] == 1:
        return 'Offensive'
    else:
        return 'Not Offensive'

In [16]:
get_prediction('you are so nice' , dt , cv)

'Not Offensive'

In [53]:
### GUI testing

In [54]:
import tkinter as tk

In [56]:
window = tk.Tk()

In [14]:
dt = pickle.load(open('decision_tree.pkl','rb'))

## Deep Learning Try

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [18]:
from sklearn.metrics import accuracy_score, log_loss
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [15]:
train_df , test_df = train_test_split(more_data,test_size = 0.2 , random_state = 23)

In [33]:
token = Tokenizer(num_words = None , char_level = False)
token.fit_on_texts(more_data.text)

In [34]:
train_seq = pad_sequences(token.texts_to_sequences(train_df.text),maxlen = 50)
test_seq = pad_sequences(token.texts_to_sequences(test_df.text),maxlen = 50)

In [35]:
train_lab = np.array(train_df.is_offensive) 
test_lab = np.array(test_df.is_offensive) 

In [37]:
emb = Embedding(len(token.word_index)+1,128,input_length = 50)
dense = Dense(1,activation = 'sigmoid')
model = Sequential()
model.add(emb)
model.add(Flatten())
model.add(dense)
model.compile(optimizer = 'sgd',loss = 'binary_crossentropy',metrics = ['accuracy'])
print(model.summary())
model.fit(train_seq,train_lab,epochs = 5)
loss,accuracy = model.evaluate(test_seq,test_lab)
print(round(accuracy * 100,2),model)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 128)           27719552  
_________________________________________________________________
flatten_5 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6401      
Total params: 27,725,953
Trainable params: 27,725,953
Non-trainable params: 0
_________________________________________________________________
None


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
147480/147480 [==============================] - 514s 3ms/step - loss: 0.4692 - accuracy: 0.7995
Epoch 2/5
147480/147480 [==============================] - 506s 3ms/step - loss: 0.4194 - accuracy: 0.8013
Epoch 3/5
147480/147480 [==============================] - 485s 3ms/step - loss: 0.3744 - accuracy: 0.8362
Epoch 4/5
147480/147480 [==============================] - 486s 3ms/step - loss: 0.3240 - accuracy: 0.8733
Epoch 5/5
36870/36870 [==============================] - 1s 20us/step
89.89 <keras.engine.sequential.Sequential object at 0x7fd53501c390>


In [40]:
names = ['I hope you die']
name_token = pad_sequences(token.texts_to_sequences(names),maxlen = 50)
pred = model.predict_classes(name_token)

In [41]:
pred

array([[0]], dtype=int32)

## OPEN AI TEST

In [36]:
import openai
openai.api_key = "sk-AxNaQumDedma7T3Kr98vS0UZ83OP037TMqSvBbQD"
response = openai.Completion.create(engine="davinci", prompt="Most people think machines will take their jobs", max_tokens=50)

In [37]:
response['choices'][0]['text']

" and make them worse. Contrary to popular opinion, the next generation may see that as a good thing. I wrote about the changing paradigm in Ask' is becoming part of our culture -- and we're going to learn that 'the network is stronger than"